In [ ]:
import pandas as pd

In [ ]:
# Import groups
tm = pd.read_csv('Step3Output_with_group.csv',
                 sep=',',
                 header=0,
                 index_col=False,
                 na_filter=False,
                 dtype=object)

tm = tm[['RecordID', 'group']]

In [ ]:
# Import metainventory
df = pd.read_csv('https://raw.githubusercontent.com/lisavirginia/clinical-abbreviations/master/code/Step3Output.csv',
                 sep='|',
                 header=0,
                 index_col=False,
                 na_filter=False,
                 dtype=object)

In [ ]:
# Merge
df = pd.merge(df, tm, on='RecordID')
df = df.sort_values(by=['group'])
df = df.reset_index(drop=True)
df.shape

In [ ]:
df.head(5)

In [ ]:
# Assign group ID
assignment = 1
for index, row in df.iterrows():
    if index == 0:
        df['GroupID'].iat[index] = assignment
    elif df['group'].at[index] == '0.0':
        assignment += 1
        df['GroupID'].iat[index] = assignment
    elif df['group'].at[index] == df['group'].at[index-1]:
        df['GroupID'].iat[index] = assignment
    else:
        assignment += 1
        df['GroupID'].iat[index] = assignment

In [ ]:
# Format group ID
df['GroupID'] = 'G' + (df.GroupID.map('{:06}'.format))
df.head(5)

In [ ]:
# Sort
df = df.sort_values(by=['RecordID'])
df = df.reset_index(drop=True)
df.head(5)

In [ ]:
# Drop extra column
df = df.drop(columns=['group'])
df.shape

In [ ]:
# Export
df.to_csv('Step4Output.csv',
          index=False,
          header=True,
          sep='|')